# Explaining the Predictions of a Convolutional Neural Network on Head CT Images

## Task

Explain the predictions of a Convolutional Neural Network (CNN) trained to predict the presence of intracranial hemorrhage on the RSNA 2019 Brain CT Hemorrhage Challenge dataset. Data is publicly available at [https://www.kaggle.com/competitions/rsna-intracranial-hemorrhage-detection/data](https://www.kaggle.com/competitions/rsna-intracranial-hemorrhage-detection/data).

## Requirements

1. Basic understanding of machine learning and deep learning.
2. Programming in Python.

## Learning objectives

1. Load a pre-trained classifier in PyTorch.
2. Use the classifier to predict the presence of hemorrhage in test images.
3. Explain the predictions of the classifier to detect intracranial hemorrhage.

## Acknowledgements

This Jupyter Notebook was based on code by Jacopo Teneggi ([jtenegg1@jhu.edu](mailto:jtenegg1@jhu.edu)).

## Load the pretrained model

This step requires PyTorch. Follow the instructions at [https://pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/) to install PyTorch. Note that, although not required, this Jupyter Notebook supports execution on a GPU.

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.set_grad_enabled(False)
model = torch.hub.load(
    "Sulam-Group/h-shap:jaco/siim-ml-tutorial",
    "rsnahemorrhagenet",
    pretrained=True,
    trust_repo="check",
)
model = model.to(device)
model.eval()

## Use the Model to Predict on Images

In [ ]:
import io
import requests
import numpy as np
import matplotlib.pyplot as plt


def window(img, WL, WW):
    image_min = WL - WW // 2
    image_max = WL + WW // 2
    img[img < image_min] = image_min
    img[img > image_max] = image_max

    img = (img - image_min) / (image_max - image_min)

    return img


# Visualize the images